In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt

# Load your data
data = pd.read_csv('../datasets/energydata_complete.csv', index_col=0, parse_dates=True)

# Check stationarity for each variable
def check_stationarity(series, significance=0.05):
    result = adfuller(series)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print('Stationary' if result[1] < significance else 'Non-stationary')

# Apply to each column
# for column in data.columns:
#     print(f"\nStationarity Test for {column}")
#     check_stationarity(data[column])

# Difference data if needed
data_diff = data.diff().dropna()

# Split data into train and test sets (last 30 periods for testing)
train_size = len(data_diff) - 30
train, test = data_diff.iloc[:train_size], data_diff.iloc[train_size:]
original_test = data.iloc[train_size:]  # For comparison in original scale
# Create and fit the VAR model
model = VAR(train)

# Select order (p) using AIC
results = model.select_order()
lag_order = results.selected_orders['aic']
print(f"Suggested order (p): {lag_order}")

# Fit the model with the selected order
fitted_model = model.fit(lag_order)
print(fitted_model.summary())

# Forecast for the test period
last_train_values = train.values[-lag_order:]
forecast_diff = fitted_model.forecast(y=last_train_values, steps=len(test))
forecast_diff_df = pd.DataFrame(forecast_diff, index=test.index, columns=train.columns)

# Convert forecasts back to original scale
# If first differencing was used:
forecast_original = pd.DataFrame(index=test.index, columns=data.columns)
for col in data.columns:
    forecast_original[col] = data.iloc[train_size-1][col] + forecast_diff_df[col].cumsum()

# Plot actual vs predicted for each variable
def plot_actual_vs_predicted(original_test, forecast_original):
    variables = original_test.columns
    # n_vars = len(variables)
    
    for i, col in enumerate(variables):
        plt.figure(i+1, figsize=(15, 10))
        plt.plot(original_test.index, original_test[col], 'b-', label='Actual')
        plt.plot(original_test.index, forecast_original[col], 'r--', label='Predicted')
        plt.title(f'Actual vs Predicted: {col}')
        plt.legend()
        
        # Calculate RMSE for this variable
        rmse = sqrt(mean_squared_error(original_test[col], forecast_original[col]))
        plt.annotate(f'RMSE: {rmse:.4f}', xy=(0.05, 0.85), xycoords='axes fraction')
    
    plt.tight_layout()
    plt.show()

# Call the function
plot_actual_vs_predicted(test, forecast_diff_df)

# Plot forecast error
def plot_forecast_error(original_test, forecast_original):
    variables = original_test.columns
    n_vars = len(variables)
    
    for i, col in enumerate(variables):
        plt.figure(i+1, figsize=(15, 10))
        forecast_error = original_test[col] - forecast_original[col]
        plt.plot(original_test.index, forecast_error, 'g-')
        plt.axhline(y=0, color='r', linestyle='-')
        plt.title(f'Forecast Error: {col}')
        
        # Calculate mean error and standard deviation
        mean_error = forecast_error.mean()
        std_error = forecast_error.std()
        plt.annotate(f'Mean Error: {mean_error:.4f}\nStd Dev: {std_error:.4f}', 
                     xy=(0.05, 0.85), xycoords='axes fraction')
    
    plt.tight_layout()
    plt.show()

# Call the function
plot_forecast_error(original_test, forecast_original)

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def preprocess_data(df, target_columns, train_ratio=0.8):
    """
    Preprocess the dataset:
    - Select target columns
    - Handle missing values
    - Normalize features
    - Split into train and test sets
    """
    df = df[target_columns].dropna()
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=target_columns, index=df.index)
    
    split_idx = int(len(df) * train_ratio)
    train, test = df_scaled.iloc[:split_idx], df_scaled.iloc[split_idx:]
    
    return train, test, scaler

def train_var_model(train_data, lags=5):
    """ Train a VAR model for multivariate data """
    model = VAR()
    model_fitted = model.fit(train_data, maxlags=lags)
    return model_fitted

def train_arima_model(train_series, order=(5,1,0)):
    """ Train an ARIMA model for univariate data """
    model = ARIMA(order=order, endog=train_series)
    model_fitted = model.fit()
    return model_fitted

def make_predictions(model, train_data, test_data, is_univariate=False):
    """ Generate multi-step forecasts for VAR or ARIMA """
    if is_univariate:
        forecast = model.forecast(steps=len(test_data))
        return pd.Series(forecast, index=test_data.index, name=train_data.name)
    else:
        lag_order = model.k_ar
        forecast_input = train_data.values[-lag_order:]
        forecast = model.forecast(forecast_input, steps=len(test_data))
        return pd.DataFrame(forecast, index=test_data.index, columns=test_data.columns)

def evaluate_model(actual, predicted):
    """ Evaluate the model using RMSE for each variable """
    if isinstance(predicted, pd.Series):
        return np.sqrt(mean_squared_error(actual, predicted))
    else:
        return {col: np.sqrt(mean_squared_error(actual[col], predicted[col])) for col in actual.columns}

# Example Usage
data = pd.read_csv("../datasets/Miles_Traveled.csv", parse_dates=True, index_col=0)
# print(data.columns)

target_columns = [data.columns.tolist()[-1]]  # Replace with actual columns

train, test, scaler = preprocess_data(data, target_columns)

if len(target_columns) == 1:
    arima_model = train_arima_model(train[target_columns[0]])
    predictions = make_predictions(arima_model, train[target_columns[0]], test[target_columns[0]], is_univariate=True)
    predictions_original = scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten()
    predictions_original = pd.Series(predictions_original, index=test.index, name=target_columns[0])
else:
    var_model = train_var_model(train, lags=5)
    predictions = make_predictions(var_model, train, test)
    predictions_original = pd.DataFrame(scaler.inverse_transform(predictions), columns=target_columns, index=test.index)

# Evaluate
errors = evaluate_model(data[target_columns].loc[test.index], predictions_original)
print("RMSE:", errors)

# Plot results
plt.figure(figsize=(12,6))
for column in target_columns:
    plt.plot(data[column], label=f"Actual {column}")
    plt.plot(predictions_original[column], linestyle="dashed", label=f"Predicted {column}")
plt.legend()
plt.title("Forecast vs Actual")
plt.show()


In [16]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR, ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def preprocess_data(df, target_column, feature_columns=None, train_ratio=0.8):
    """
    Preprocess the dataset:
    - Select target and feature columns
    - Handle missing values
    - Normalize features
    - Split into train and test sets
    """
    # If there are features, include them in the dataset, otherwise just use the target column
    if feature_columns:
        df = df[[target_column] + feature_columns].dropna()
    else:
        df = df[[target_column]].dropna()

    # Scale the features and target
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=[target_column] + (feature_columns or []), index=df.index)
    
    # Split the data into training and test sets
    split_idx = int(len(df) * train_ratio)
    train, test = df_scaled.iloc[:split_idx], df_scaled.iloc[split_idx:]
    
    return train, test, scaler

def train_var_model(train_data, lags=5):
    """ Train a VAR model for multivariate data """
    model = VAR(train_data)
    model_fitted = model.fit(lags)
    return model_fitted

def train_arima_model(train_series, order=(5, 1, 0)):
    """ Train an ARIMA model for univariate data """
    model = ARIMA(train_series, order=order)
    model_fitted = model.fit()
    return model_fitted

def make_predictions(model, train_data, test_data, is_univariate=False):
    """ Generate multi-step forecasts for VAR or ARIMA """
    if is_univariate:
        forecast = model.forecast(steps=len(test_data))
        return pd.Series(forecast, index=test_data.index, name=train_data.name)
    else:
        lag_order = model.k_ar
        forecast_input = train_data.values[-lag_order:]
        forecast = model.forecast(forecast_input, steps=len(test_data))
        return pd.DataFrame(forecast, index=test_data.index, columns=test_data.columns)

def evaluate_model(actual, predicted):
    """ Evaluate the model using RMSE for each variable """
    if isinstance(predicted, pd.Series):
        return np.sqrt(mean_squared_error(actual, predicted))
    else:
        return {col: np.sqrt(mean_squared_error(actual[col], predicted[col])) for col in actual.columns}

# Example Usage
# Example Usage
data = pd.read_csv("../datasets/Month_Value_1.csv", parse_dates=True, index_col=0)
data.dropna(inplace=True)


# Define the target column and features
target_column = data.columns[-1]  # Replace with your target column name
feature_columns = []  # Replace with your feature column names

# Preprocess the data
train, test, scaler = preprocess_data(data, target_column, feature_columns)

if len(feature_columns) == 0:  # Univariate case (only y)
    # Train the ARIMA model
    arima_model = train_arima_model(train[target_column])
    
    # Make predictions
    predictions = make_predictions(arima_model, train[target_column], test[target_column], is_univariate=True)
    
    # Inverse transform predictions to the original scale
    predictions_original = scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten()
    predictions_original = pd.Series(predictions_original, index=test.index, name=target_column)
else:  # Multivariate case (y and X1, X2, ...)
    # Train the VAR model
    var_model = train_var_model(train, lags=5)
    
    # Make predictions
    predictions = make_predictions(var_model, train, test)
    
    # Inverse transform predictions to the original scale
    predictions_original = pd.DataFrame(scaler.inverse_transform(predictions), 
                                        columns=[target_column] + feature_columns, 
                                        index=test.index)

# Evaluate the model's performance (RMSE)
errors = evaluate_model(data[[target_column] + (feature_columns or [])].loc[test.index], predictions_original)
print("RMSE:", errors)

# Plot the results
plt.figure(figsize=(12,6))
plt.plot(data[target_column], label=f"Actual {target_column}")
plt.plot(predictions_original[target_column], linestyle="dashed", label=f"Predicted {target_column}")
plt.legend()
plt.title(f"{target_column} Forecast vs Actual")
plt.show()


C:\Users\raedb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\raedb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\raedb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g

ValueError: Input contains NaN.